<a href="https://colab.research.google.com/github/rssubramaniyan1/EVA8/blob/main/EVA_8__Assignment_2_dot_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Problem Statement**

Write a neural network that can take 2 inputs:


> an image from the MNIST dataset (say 5), and
> a random number between 0 and 9, (say 7)

and gives two outputs:

> the "number" that was represented by the MNIST image (predict 5)

>the "sum" of this number with the random number and the input image to the network (predict 5 + 7 = 12)

#############################################################################

# **Data generation strategy for the random numbers**

> we will generate a torch tensor with 28 columns and 28 rows with all zeros and then replace the zeros with 1s

> generate an integer between 0 and 9

> create a one hot encoded vector with 28 columns and 1 row with all zeros and then replace the zeros with 1s at the index of the integer

> multiply the random_number with the one_hot and reshape it to a [1, 28, 28] tensor with a single channel 

> the mnist data strategy is same as what was used in class/prev assignment

#############################################################################

# **Define network**

> create a network with two inputs random number and MNIST image

> The network is designed to operate independently of each other for each of the inputs.

> The first branch consists of four convolutional layers and two max pooling layers.

> The second branch consists of the same four convolutional layers and two max pooling layers.

> The output of each branch is passed through a fully connected layer,
and then the final output is obtained using a log softmax function.

# This approach allows to idependently predict the mnist and the input random number. 
# This approach has given in  the final epoch (epochs =5) an accuracy of 

> **Test set: Average loss: 0.000000, MNIST_Test_Accuracy: 9917/10000 (99.17%), Random_Number_Test_Accuracy: 10000/10000 (100.00%)** 

#############################################################################
# **Assignment Objective**
#  using the trained model, given a MNSIT image and a random number, we predict mnist and random number and print the prediction of mnist,random number and sum of the two


# import the necessary packages

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from torchsummary import summary

In [2]:

use_cuda = torch.cuda.is_available() # check if GPU is available
device = torch.device("cuda" if use_cuda else "cpu") # use GPU if available
torch.manual_seed(1) # set the random seed
batch_size = 64 # set the batch size
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}  # set the number of workers and pin memory if GPU is available


**Data generation strategy for the random numbers**

> we will generate a torch tensor with 28 columns and 28 rows with all zeros and then replace the zeros with 1s

> generate an integer between 0 and 9

> create a one hot encoded vector with 28 columns and 1 row with all zeros and then replace the zeros with 1s at the index of the integer

> multiply the random_number with the one_hot and reshape it to a [1, 28, 28] tensor with a single channel

In [3]:
def rand_number_gen():
    random_number = torch.zeros(28,28 ) # create one hot encoded vectors for the random numbers
    random_number[:,] = 1 # replace the zeros in the random_number with 1s
    integer = torch.randint(0, 10, size=(1,)) # Generate a tensor with a single random integer between 0 and 9
    one_hot = torch.zeros(28, ) # convert the integer to a one hot encoded vector
    one_hot[integer] = 1 # replace the zeros in the one_hot with 1s at the index of the integer
    random_number = (random_number * one_hot).view(1,28,28) # multiply the random_number with the one_hot and reshape it to a column vector
    random_number_label = integer.item() # create a tensor with the integer as the label
    return (random_number, random_number_label)

# create a train and test dataset of 60k and 10k random numbers and their labels

# we create a class that uses the rand_number_gen function to create a dataset of random numbers and their labels

class RandomNumberDataset():
    def __init__(self, size):
        self.size = size

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return rand_number_gen()

In [4]:
random_number_trainset = RandomNumberDataset(60000) 
random_number_testset = RandomNumberDataset(10000) 

The random_number_trainset and random_number_testset are now ready to be used in a data loader to inherit the properties of the torch.utils.data.DataLoader class

In [5]:
train_loader_1 = torch.utils.data.DataLoader(random_number_trainset, batch_size=batch_size, shuffle=True, **kwargs)
test_loader_1 = torch.utils.data.DataLoader(random_number_testset, batch_size=batch_size, shuffle=True, **kwargs)


# Data generation strategy for the mnist images

>  This is exactly the same as was discussed in class and used in prev. assignment

In [6]:
#  download MNIST train dataset
train_set = torchvision.datasets.MNIST('./data', train=True, download=True,
                                       transform=torchvision.transforms.Compose([
                                           torchvision.transforms.ToTensor(),
                                           torchvision.transforms.Normalize((0.1307,), (0.3081,))
                                       ]))

#  download MNIST test dataset
test_set = torchvision.datasets.MNIST('./data', train=False, download=True,
                                      transform=torchvision.transforms.Compose([
                                          torchvision.transforms.ToTensor(),
                                          torchvision.transforms.Normalize((0.1307,), (0.3081,))
                                      ]))


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Create a data loader for the train set and test set of the mnist images


In [7]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, **kwargs)

# Create a network with two inputs random number and MNIST image

> The network is designed to operate independently of each other for each of the inputs.

>  The first branch consists of four convolutional layers and two max pooling layers.

> The second branch consists of the same four convolutional layers and two max pooling layers.

>  The output of each branch is passed through a fully connected layer,and then the final output is obtained using a log softmax function.

In [8]:
class twoinputnetwork(nn.Module):
    def __init__(self):
        super(twoinputnetwork, self).__init__()

        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) # batch size 16, 28x28 input; parameters 16x3x3x1 + 16 = 160
        self.conv1_y = nn.Conv2d(1, 16, 3, padding=1) # batch size 16, 28x28 input; parameters 16x3x3x1 + 16 = 160
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1) # batch size 16, 28x28
        self.conv2_y = nn.Conv2d(16, 16, 3, padding=1) # batch size 16, 28x28
        self.pool1 = nn.MaxPool2d(2, 2) # batch size 16, 14x14
        self.pool1_y = nn.MaxPool2d(2, 2) # batch size 16, 14x14
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1) # batch size 16, 14x14
        self.conv3_y = nn.Conv2d(16, 16, 3, padding=1) # batch size 16, 14x14
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1) # batch size 32, 14x14
        self.conv4_y = nn.Conv2d(16, 32, 3, padding=1) # batch size 32, 14x14
        self.pool2 = nn.MaxPool2d(2, 2) # batch size 32, 7x7
        self.pool2_y = nn.MaxPool2d(2, 2) # batch size 32, 7x7
        # a fully connected layer with 32 outputs
        self.fc1 = nn.Linear(32 * 7 * 7, 10) # batch size 10
        self.fc1_y = nn.Linear(32 * 7 * 7, 10) # batch size 10

    def forward(self, x, y):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) # 28x28x16
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))) # 14x14x32
        y = self.pool1_y(F.relu(self.conv2_y(self.conv1_y(y))))
        y = self.pool2_y(F.relu(self.conv4_y(self.conv3_y(y))))
        x = x.view(-1, 32 * 7 * 7) # flatten the tensor
        y = y.view(-1, 32 * 7 * 7) # flatten the tensor
        x = self.fc1(x)
        y = self.fc1_y(y)
        return F.log_softmax(x, dim=1), F.log_softmax(y, dim=1)

In [9]:
model = twoinputnetwork().to(device) # define the network and move it to the device
summary(model, [(1, 28, 28), (1, 28, 28)]) # print the network summary

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
            Conv2d-2           [-1, 16, 28, 28]           2,320
         MaxPool2d-3           [-1, 16, 14, 14]               0
            Conv2d-4           [-1, 16, 14, 14]           2,320
            Conv2d-5           [-1, 32, 14, 14]           4,640
         MaxPool2d-6             [-1, 32, 7, 7]               0
            Conv2d-7           [-1, 16, 28, 28]             160
            Conv2d-8           [-1, 16, 28, 28]           2,320
         MaxPool2d-9           [-1, 16, 14, 14]               0
           Conv2d-10           [-1, 16, 14, 14]           2,320
           Conv2d-11           [-1, 32, 14, 14]           4,640
        MaxPool2d-12             [-1, 32, 7, 7]               0
           Linear-13                   [-1, 10]          15,690
           Linear-14                   

# Since the network has two inputs and operate independently of each other, we define two optimizers for the two inputs random number and MNIST image

In [10]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # optimizer for mnist
optimizer_y = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # optimizer for random_number

# Define the loss function for the random number and MNIST image

In [11]:
criterion = nn.CrossEntropyLoss()

#**Training Function**

In [12]:
def compute_train_accuracy(model, data_loader, data_loader_1, device):
    model.eval()
    correct = 0
    correct_y = 0
    total = 0
    total_y = 0
    # pbar and pbar_1 are just iterators that iterate over the batches in train_loader and train_loader_1 respectively.
    # enumerate is used to loop over the batches and also get the index of the current batch. The zip function is used
    # to combine the two iterators and create an iterator that returns tuples of two batches,
    # one from each of the data loaders.
    pbar = tqdm(train_loader) # loading the progress bar for the mnist train set
    pbar_1 = tqdm(train_loader_1) # loading the progress bar for the random number train set
    for batch_idx,((data, label), (data_y, label_y)) in enumerate(zip(pbar, pbar_1)):
        data = data.to(device) # move the data to the device
        label = label.to(device)    # move the label to the device
        data_y = data_y.to(device)  # move the data to the device
        label_y = label_y.to(device)   # move the label to the device
        optimizer.zero_grad()   # clear the gradients
        optimizer_y.zero_grad() # clear the gradients
        output, output_y = model(data, data_y)  # get the output from the model for the mnist image and random number
        loss = criterion(output, label) # calculate the loss for the mnist image
        loss_y = criterion(output_y, label_y) # calculate the loss for the random number
        loss.backward() # backpropagate the loss for the mnist image
        loss_y.backward() # backpropagate the loss for the random number
        optimizer.step() # update the weights for the mnist image
        optimizer_y.step() # update the weights for the random number
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        pred_y = output_y.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(label.view_as(pred)).sum().item() # calculate the number of correct predictions for the mnist image
        correct_y += pred_y.eq(label_y.view_as(pred_y)).sum().item() # calculate the number of correct predictions for the random number
        total += len(data) # calculate the total number of predictions for the mnist image
        total_y += len(data_y) # calculate the total number of predictions for the random number
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}') # update the progress bar for the mnist image and print the loss
        pbar_1.set_description(desc= f'loss_y={loss_y.item()} batch_id={batch_idx}') # update the progress bar for the random number and print the loss
    return 100. * correct / total, 100. * correct_y / total_y # return the accuracy for the mnist image and random number


#**Test Function**

In [13]:
# define the test function to test the model
def compute_test_accuracy(model, data_loader, data_loader_1, device):
    model.eval()
    test_loss = 0
    correct = 0
    correct_y = 0
    total = 0
    total_y = 0
    pbar = tqdm(test_loader)
    pbar_1 = tqdm(test_loader_1)
    with torch.no_grad(): # disable the gradient calculation for the test set
        for batch_idx,((data, label), (data_y, label_y)) in enumerate(zip(pbar, pbar_1)): # iterate over the test set
            data = data.to(device)  # move the data to the device
            label = label.to(device) # move the label to the device
            data_y = data_y.to(device) # move the data to the device
            label_y = label_y.to(device) # move the label to the device
            output, output_y = model(data, data_y) # get the output from the model for the mnist image and random number
            test_loss += criterion(output, label).item()  # sum up batch loss
            test_loss += criterion(output_y, label_y).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            pred_y = output_y.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(label.view_as(pred)).sum().item() # calculate the number of correct predictions for the mnist image for each batch of test set
            correct_y += pred_y.eq(label_y.view_as(pred_y)).sum().item() # calculate the number of correct predictions for the random number for each batch of test set
            total += len(data) # calculate the total number of predictions for the mnist image
            total_y += len(data_y) # calculate the total number of predictions for the random number
            pbar.set_description(desc= f'loss={test_loss} batch_id={batch_idx}') # update the progress bar for the mnist image and print the loss
            pbar_1.set_description(desc= f'loss_y={test_loss} batch_id={batch_idx}') # update the progress bar for the random number and print the loss
    test_loss /= len(data_loader.dataset)
    test_loss /= len(data_loader_1.dataset)

    print('Test set: Average loss: {:.6f}, MNIST_Test_Accuracy: {}/{} ({:.2f}%), Random_Number_Test_Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, total,
        100. * correct / total, correct_y, total_y,
        100. * correct_y / total_y))


#**Train the model for 5 epochs**

In [14]:
for epoch in range(1, 5 + 1):
    accuracy, accuracy_y = compute_train_accuracy(model, train_loader, train_loader_1, device)
    print('Train set: Accuracy: {:.2f}%'.format(accuracy))
    print('Train set: Accuracy: {:.2f}%'.format(accuracy_y))
    compute_test_accuracy(model, test_loader, test_loader_1, device)

loss_y=8.530898867320502e-07 batch_id=937: 100%|█████████▉| 937/938 [00:36<00:00, 25.61it/s]


Train set: Accuracy: 93.65%
Train set: Accuracy: 96.87%


loss_y=7.778550664708348 batch_id=156:  99%|█████████▉| 156/157 [00:05<00:00, 26.19it/s]


Test set: Average loss: 0.000000, MNIST_Test_Accuracy: 9831/10000 (98.31%), Random_Number_Test_Accuracy: 10000/10000 (100.00%)


loss_y=3.6507813661046384e-07 batch_id=937: 100%|█████████▉| 937/938 [00:47<00:00, 19.92it/s]


...

Test set: Average loss: 0.000000, MNIST_Test_Accuracy: 9891/10000 (98.91%), Random_Number_Test_Accuracy: 10000/10000 (100.00%)


loss_y=1.8626447229053156e-07 batch_id=937: 100%|█████████▉| 937/938 [00:38<00:00, 24.33it/s]


Train set: Accuracy: 99.12%
Train set: Accuracy: 100.00%


loss_y=4.958580429162808 batch_id=156:  99%|█████████▉| 156/157 [00:05<00:00, 27.81it/s]

Test set: Average loss: 0.000000, MNIST_Test_Accuracy: 9904/10000 (99.04%), Random_Number_Test_Accuracy: 10000/10000 (100.00%)


#**Assignment Solution**

Given a MNSIT image and a random number, predict mnist and random number using the trained model get the model prediction for the given image and random number and print the prediction of mnist,random number and sum of the two

In [15]:
p= 10 # using this value of p we can select an arbitrary mnist image and random number from the respective test data. since the max of test data is 10k enter value for p : 0 < = p < 10000

In [16]:
data,label = test_set[p]
data_y,label_y = random_number_testset[p]
data = data.to(device)
label = torch.tensor(label).to(device)
data_y = data_y.to(device)
label_y = torch.tensor(label_y).to(device)


In [17]:
print("Ground Truth MNIST: ", label)

Ground Truth MNIST:  tensor(0, device='cuda:0')


In [18]:
print("Ground Truth Random Number: ", label_y)

Ground Truth Random Number:  tensor(3, device='cuda:0')


In [19]:
output, output_y = model(data, data_y)
pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
pred_y = output_y.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
print("Predicted MNIST: ", pred)
print("Predicted Random Number: ", pred_y)
print("Sum of MNIST and Random Number: ", pred + pred_y)

Predicted MNIST:  tensor([[0]], device='cuda:0')
Predicted Random Number:  tensor([[3]], device='cuda:0')
Sum of MNIST and Random Number:  tensor([[3]], device='cuda:0')
